In this part, we'll try to plot the time-course of 1 IC extracted from the EEG of 1 subject on 1 trial.

# First let's extract the time scale data and the time-course data from the .mat file:

In [1]:
import h5py
import numpy as np
# import xarray as xr
# from utilities import flatten_dict

In [2]:
path = "E:/timot/Documents/1 - Centrale Marseille/0.5 - Semestre S8/Stage/NIC_250819/FCK_LOCKED_IC_JYOTIKA_250819.mat"

mat_file = h5py.File(path, "r")
cells_refs = mat_file['FCK_LOCKED_IC_JYOTIKA']
n_IC, n_subj = cells_refs.shape

Here we define the functions that extract the data from the .mat file:

In [3]:
def _load_raw_timecourse_256Hz(IC, subj):
    cell = mat_file[cells_refs[IC, subj]]
    raw_timecourse_256Hz = cell['raw_timecourse_256Hz'][:]
    raw_timecourse_256Hz = np.transpose(raw_timecourse_256Hz)
    return raw_timecourse_256Hz

In [4]:
def _load_time_256Hz(IC,subj):
    cell = mat_file[cells_refs[IC, subj]]
    time_256Hz = cell['time_256Hz'][:]
    time_256Hz = np.transpose(time_256Hz)[0]
    return time_256Hz

In [5]:
def _load_freq(IC,subj):
    cell = mat_file[cells_refs[IC, subj]]
    freq = cell['freq'][:]
    freq = np.transpose(freq)[0]
    return freq

Here put in one dictionary every `'raw_timecourse_256Hz subject{subj+1}, IC{IC+1}'`, the `'time_axis'`, and the `'freq_axis'`:

In [6]:
data = {} 
for IC in range(n_IC):
    for subj in range(n_subj):
        try:
            data[f'raw_timecourse_256Hz subject{subj+1}, IC{IC+1}'] = _load_raw_timecourse_256Hz(IC, subj)
        except:
            print(f'The independent component IC{IC+1} of the subject {subj+1} is not in the .mat file.')

The independent component IC1 of the subject 20 is not in the .mat file.
The independent component IC1 of the subject 21 is not in the .mat file.
The independent component IC2 of the subject 5 is not in the .mat file.
The independent component IC2 of the subject 9 is not in the .mat file.
The independent component IC3 of the subject 1 is not in the .mat file.
The independent component IC3 of the subject 17 is not in the .mat file.
The independent component IC3 of the subject 23 is not in the .mat file.
The independent component IC4 of the subject 12 is not in the .mat file.


In [7]:
data['freq_axis'] = _load_freq(0, 0)
data['time_axis'] = _load_time_256Hz(0, 0)

In [8]:
n_trials = data['raw_timecourse_256Hz subject1, IC1'].shape[0]

# Now we'll implement the HMM method.

In [9]:
from sklearn.decomposition import PCA
from hmmlearn import hmm
import scipy.signal as signal

In [10]:
# La fonction embedx copie le vecteur `x` len(lags) fois dans `xe`
# avec des décalages entre lags[0] et lags[-1] (on implémente le délai temporel pour le HMM)

def embedx(x, lags):
    
    Xe = np.zeros((x.shape[1], x.shape[0],  len(lags)))

    for l in range(len(lags)):
        Xe[:, :, l] = np.roll(x, lags[l], axis=0).swapaxes(0, 1)

    # Remove edges
    valid = np.ones((x.shape[0], 1), dtype=np.int8)
    valid[:np.abs(np.min(lags)), :] = 0
    valid[-np.abs(np.max(lags)):, :] = 0

    Xe = Xe[:, valid[:, 0] == 1, :]

    return Xe, valid


# La fonction hmm_tde trouve les paramètres du HMM

def hmm_tde(data: np.array, lags, n_states=3, n_iter=100, n_components=8, 
            covariance_type='full', model_type='GMMHMM', tol=0.01, n_mix=1, **kwargs):
    
    # Embed time serie
    xe, valid = embedx(data, lags)

    pca = PCA(n_components=n_components)
    y = pca.fit_transform(xe[0, :, :])
    
    if model_type=='GMMHMM':
        model = hmm.GMMHMM(n_components=n_states, n_iter=n_iter,
                            covariance_type=covariance_type, tol=tol, n_mix=n_mix, **kwargs)
        
    elif model_type=='GaussianHMM':
        model = hmm.GaussianHMM(n_components=n_states, n_iter=n_iter,
                            covariance_type=covariance_type, tol=tol, **kwargs)
        
    elif model_type=='MultinomialHMM':
        model = hmm.MultinomialHMM(n_components=n_states, n_iter=n_iter, tol=tol, **kwargs)
    
    else: 
        return "Non-exixting model_type. Please choose 'GMMHMM' or 'GaussianHMM' or 'MultinomialHMM'. default='GMMHMM'"
        
    model.fit(y)
    gamma = model.predict_proba(y)
    
    return gamma, model, xe

In [11]:
# The data we want to analyse with the HMM method
subj=1
IC=1
trial=1
x = data[f'raw_timecourse_256Hz subject{subj}, IC{IC}'][trial-1, :].reshape(-1, 1)

In [12]:
# The parameters we change to hope for some results
lags = np.arange(-29, 29)
n_iter=1000
n_states=3    # nb d'états du modèle
n_components=8     # nb de composantes principales à extraire
covariance_type='diag'
model_type='GMMHMM'
tol=0.001
n_mix=3

save=False

covars_prior=None
covars_weight=None
means_prior=0.0
means_weight=0.0

In [13]:
# Finding the model
gamma, model, xe = hmm_tde(x, lags, n_iter=n_iter, n_states=n_states, n_components=n_components, 
                           covariance_type=covariance_type, model_type=model_type, tol=tol, n_mix=n_mix)

## Burst duration and count

In [14]:
gamma_bin = gamma[:,1]>0.6
time = data['time_axis'][np.abs(np.min(lags)):-np.abs(np.max(lags))]

def burst_duration(gamma_bin, time):
    m=0
    for b in gamma_bin:
        if b==True:
            m+=1
    mean=m*(time[-1]-time[0])/time.shape
    return mean[0]

print(time[-1]-time[0], time.shape)
burst_duration(gamma_bin, time)

6.77734375 (1736,)


1.9988479262672811

In [15]:
def burst_count(gamma_bin, time):
    nb=0
    for i in range(gamma_bin.shape[0]-1):
        if gamma_bin[i]==True and gamma_bin[i+1]==False: 
            nb+=1
    t=time[-1]-time[0]
    return nb/t

burst_count(gamma_bin, time)

1.4755043227665705